# Visualizer sandbox

Getting the muselsl visualizer template to work with brainflow input.

In [2]:
from pprint import pprint

import brainflow
from brainflow.board_shim import BoardShim, BoardIds

In [3]:
board_id = BoardIds.MUSE_2_BOARD.value
board_desc = BoardShim.get_board_descr(board_id)
pprint(board_desc)

{'accel_channels': [5, 6, 7],
 'eeg_channels': [1, 2, 3, 4],
 'eeg_names': 'TP9,Fp1,Fp2,TP10',
 'gyro_channels': [8, 9, 10],
 'marker_channel': 15,
 'name': 'Muse2',
 'num_rows': 16,
 'package_num_channel': 0,
 'ppg_channels': [11, 12, 13],
 'sampling_rate': 256,
 'timestamp_channel': 14}


In [6]:
BoardShim.get_current_board_data(board_id)

TypeError: get_current_board_data() missing 1 required positional argument: 'num_samples'